In [1]:
import requests
import pandas as pd
import json

In [126]:
# Set your Steam API key
api_key = "A9283666CBEBA01186FF7EB5D8F80120"

steam_id = 76561198022981376

# Set the API endpoint and parameters
url = f"http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key={api_key}&steamid={steam_id}&format=json"

# Make the API call
response = requests.get(url)

# Parse the response
data = json.loads(response.text)

# Extract the user's game data

games = data["response"]['games']


# Note: playtime_forever is in minutes
game_data = []


# Loop through the games and extract the appid and playtime_forever

for game in games:

    game_data.append({"appid": game["appid"], "playtime_forever": game["playtime_forever"]})

dict = {'steam_id':steam_id,
        'game_data':game_data}
dict

{'steam_id': 76561198022981376,
 'game_data': [{'appid': 240, 'playtime_forever': 61},
  {'appid': 4000, 'playtime_forever': 123},
  {'appid': 4760, 'playtime_forever': 0},
  {'appid': 15620, 'playtime_forever': 0},
  {'appid': 20570, 'playtime_forever': 0},
  {'appid': 56400, 'playtime_forever': 0},
  {'appid': 105600, 'playtime_forever': 872},
  {'appid': 55150, 'playtime_forever': 174},
  {'appid': 8930, 'playtime_forever': 2833},
  {'appid': 22380, 'playtime_forever': 2033},
  {'appid': 200710, 'playtime_forever': 109},
  {'appid': 238960, 'playtime_forever': 0},
  {'appid': 8870, 'playtime_forever': 82},
  {'appid': 39210, 'playtime_forever': 16451},
  {'appid': 211820, 'playtime_forever': 544},
  {'appid': 367540, 'playtime_forever': 0},
  {'appid': 550, 'playtime_forever': 0},
  {'appid': 65980, 'playtime_forever': 0},
  {'appid': 730, 'playtime_forever': 0},
  {'appid': 333930, 'playtime_forever': 0},
  {'appid': 346110, 'playtime_forever': 0},
  {'appid': 407530, 'playtime_for

In [116]:
def get_app_list():
    url = "https://api.steampowered.com/ISteamApps/GetAppList/v2/"
    response = requests.get(url)
    app_list = response.json()
    return app_list['applist']['apps']

In [137]:
game_names = get_app_list()

appid_name_map = {game['appid']: game['name'] for game in game_names}


for game in dict['game_data']:

    if game['appid'] in appid_name_map:

        game['name'] = appid_name_map[game['appid']]


dict

{'steam_id': 76561198022981376,
 'game_data': [{'appid': 240,
   'playtime_forever': 61,
   'name': 'Counter-Strike: Source'},
  {'appid': 4000, 'playtime_forever': 123, 'name': "Garry's Mod"},
  {'appid': 4760, 'playtime_forever': 0, 'name': 'Rome: Total War'},
  {'appid': 15620,
   'playtime_forever': 0,
   'name': 'Warhammer 40,000: Dawn of War II'},
  {'appid': 20570,
   'playtime_forever': 0,
   'name': 'Warhammer 40,000: Dawn of War II - Chaos Rising'},
  {'appid': 56400,
   'playtime_forever': 0,
   'name': 'Warhammer 40,000: Dawn of War II - Retribution'},
  {'appid': 105600, 'playtime_forever': 872, 'name': 'Terraria'},
  {'appid': 55150,
   'playtime_forever': 174,
   'name': 'Warhammer 40,000: Space Marine - Anniversary Edition'},
  {'appid': 8930,
   'playtime_forever': 2833,
   'name': "Sid Meier's Civilization V"},
  {'appid': 22380, 'playtime_forever': 2033, 'name': 'Fallout: New Vegas'},
  {'appid': 200710, 'playtime_forever': 109, 'name': 'Torchlight II'},
  {'appid': 

In [145]:
# Extract the game data into a list of dictionaries

game_data = dict['game_data']


# Create a list to store the column names (game names)

column_names = [game['name'] for game in game_data]

print(column_names)
# Create a list to store the playtime values

playtime_values = [game['playtime_forever'] for game in game_data]

print(playtime_values)
# Create a dictionary to store the data for the DataFrame

df_data = {'steam_id': [dict['steam_id']] + playtime_values}


# Create the DataFrame

df = pd.DataFrame(df_data, columns=['steam_id'] + column_names)




['Counter-Strike: Source', "Garry's Mod", 'Rome: Total War', 'Warhammer 40,000: Dawn of War II', 'Warhammer 40,000: Dawn of War II - Chaos Rising', 'Warhammer 40,000: Dawn of War II - Retribution', 'Terraria', 'Warhammer 40,000: Space Marine - Anniversary Edition', "Sid Meier's Civilization V", 'Fallout: New Vegas', 'Torchlight II', 'Path of Exile', 'BioShock Infinite', 'FINAL FANTASY XIV Online', 'Starbound', 'Starbound - Unstable', 'Left 4 Dead 2', "Sid Meier's Civilization: Beyond Earth", 'Counter-Strike 2', 'Dirty Bomb', 'ARK: Survival Evolved', 'ARK: Survival Of The Fittest', 'Skyrim Script Extender (SKSE)', 'Hollow Knight', 'DARK SOULS™ III', "Sid Meier's Civilization VI", 'The Witcher 3: Wild Hunt', 'Deep Rock Galactic', 'Far Cry 5', 'HELLDIVERS™ 2', 'Wolfenstein II: The New Colossus', 'Total War: WARHAMMER II', 'Blasphemous', 'Warhammer: Vermintide 2', 'Valheim', 'Aimlabs', 'Sekiro™: Shadows Die Twice', 'Among Us', 'Destiny 2', 'Apex Legends', 'Hades', 'Borderlands 3', 'Mass Ef

In [148]:
game_data = dict['game_data']


# Create a dictionary to store the data for the DataFrame

df_data = {'steam_id': [dict['steam_id']]}


# Add the playtime values to the dictionary

for game in game_data:

    df_data[game['name']] = [game['playtime_forever']]


# Create the DataFrame

df = pd.DataFrame(df_data)


df

,steam_id,Counter-Strike: Source,Garry's Mod,Rome: Total War,"Warhammer 40,000: Dawn of War II","Warhammer 40,000: Dawn of War II - Chaos Rising","Warhammer 40,000: Dawn of War II - Retribution",Terraria,"Warhammer 40,000: Space Marine - Anniversary Edition",Sid Meier's Civilization V,...,Hades,Borderlands 3,Mass Effect™ Legendary Edition,KovaaK's,Halo Infinite,ELDEN RING,The Elder Scrolls V: Skyrim Special Edition,Call of Duty®,Starfield,Baldur's Gate 3
0,76561198022981376,61,123,0,0,0,0,872,174,2833,...,3999,61,0,632,703,18715,93,3053,3084,1759
